# Part 6.1 - Prepare Dataframe
---

### Papers Past Topic Modeling
<br/>


Ben Faulks - bmf43@uclive.ac.nz

Xiandong Cai - xca24@uclive.ac.nz

Yujie Cui - ycu23@uclive.ac.nz

In [1]:
import sys, gc
sys.path.insert(0, '../utils')
from utils import conf_pyspark, load_dataset
from utils_preplot import preplot, load_doctopic
import pandas as pd
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_colwidth', 120)
from pyspark.sql import functions as F
from pyspark.sql.types import *

# intiate PySpark
sc, spark = conf_pyspark()

sc

[('spark.driver.host', 'x99.hub'),
 ('spark.app.name', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.memory', '62g'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '44580'),
 ('spark.app.id', 'local-1548489116478'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.cores', '6'),
 ('spark.driver.maxResultSize', '4g')]


<SparkContext master=local[*] appName=local>

**This part we generate the dataframe that part 6.2 will be used:**
* dominant topic per year dataframe;
* average topic weight per year dataframe;
* dominant topic per month dataframe (only for WWI analysis);
* average topic weight per month dataframe (only for WWI analysis);

**After training topic models, we got the topic words list (**`topicKeys.txt`**) and the doc-topic matrix (**`docTopics.txt`**). The doc-topic matrix is a docmument based file which means we could connect the topic weights in the matrix and the metadata (date, region, publisher) in the meta dataset, then the combined dataset could apply to accomplish many data mining or statistical works, for instance, the granularity of time can achieve 1 day, which could perform time series analsys. Now we load those data to generate dataframe for analysys and visualization.**

## 1 Prepare Dataframe for Train Set

### 1.1 Load data

**Load metadata ("id", "region" and "date"):**

In [2]:
df_meta = load_dataset('meta', spark).select(F.col('id').alias('id_'), F.col('region'), F.col('date'))

In [3]:
df_meta.limit(5).toPandas().head()

,id_,region,date
0,1854215,Auckland,1862-06-14
1,1854221,Auckland,1862-06-14
2,1854224,Auckland,1862-07-03
3,1854233,Auckland,1863-08-01
4,1854245,Auckland,1863-09-01


**Load topic words list:**

In [3]:
path = r'../models/train/topicKeys.txt'

data_schema = StructType([
    StructField('topic', IntegerType()),
    StructField('weight_', FloatType()),
    StructField('words', StringType())
])

df_topics = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "false")
    .option("delimiter", "\t")
    .schema(data_schema)
    .load(path)
)
topic_number = df_topics.count()

In [5]:
print('Shape of dataframe: ({}, {})'.format(topic_number, len(df_topics.columns)))
df_topics.limit(10).toPandas().head(10)

Shape of dataframe: (200, 3)


,topic,weight_,words
0,0,0.00194,theatre tonight exhibition universal approved adults censor recommended comedy tomorrow night plans dance george dan...
1,1,0.00774,day people procession mayor city crowd town large governor reception band morning great afternoon excellency visit p...
2,2,0.00689,election electors candidate candidates vote poll district votes elections returning elected officer council seat ele...
3,3,0.00764,oamaru aro thames dunedin north otago street waitaki john zealand town timaru kurow fleming good tyne sumpter hold g...
4,4,0.00998,christchurch timaru ashburton street canterbury akaroa lyttelton east road south john colombo bay temuka thomas burn...
5,5,0.03908,association press received telegraph copyright london united electric july cable sydney august june april march aust...
6,6,0.00676,price lid yard sale ladies white prices pair silk special black usual wide worth navy blouses dozen dress shades good
7,7,0.01094,police man arrested men murder found made shot night house crime arrest named discovered revolver fired killed priso...
8,8,0.01595,business public begs orders stock notice attention inform premises street prices goods general made patronage custom...
9,9,0.00375,black ladies white men's fancy hats silk shirts cloth yard tweed boys wool cotton dress blankets jackets trousers fr...


**Load doc-topic matrix:**

In [6]:
path = r'../models/train/docTopics.txt'

# generate new column names
columns = [str(x) for x in list(range(topic_number))]
columns.insert(0, 'id')
columns.insert(0, 'index')

# load data
df_doctopic = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "true")
    .option("delimiter", "\t")
    .load(path)
)

# change columns name and drop # column which is table index and useless
df_doctopic = df_doctopic.toDF(*columns)

In [7]:
print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

df_doctopic.limit(5).toPandas().head()

Shape of dataframe: (1814086, 202)


,index,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,...,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,0,1854215,0.000033,0.000132,0.000118,0.000131,0.000171,0.000668,0.000116,0.000187,0.000273,0.000064,0.000112,0.000099,0.000060,0.000059,0.000085,0.000147,0.000094,0.000080,0.000199,0.000119,0.000104,0.000062,0.000165,0.000138,0.000107,0.000165,0.000194,0.000101,0.017802,0.000194,0.000138,0.000077,0.000127,0.000134,0.000149,0.000070,0.000114,0.000213,0.000095,0.000208,0.000109,0.000036,0.000136,0.000133,0.000133,0.000115,0.000147,0.000116,0.000129,0.000086,0.000236,0.000124,0.000107,0.000105,0.000113,0.000115,0.000103,0.000175,...,0.000030,0.000227,0.000114,0.000099,0.000075,0.000087,0.000079,0.000069,0.017168,0.000104,0.000114,0.000138,0.000066,0.000118,0.000102,0.000109,0.000042,0.000033,0.000024,0.000101,0.000105,0.000043,0.000095,0.000189,0.000315,0.000096,0.000114,0.000144,0.000021,0.000219,0.273583,0.000145,0.000116,0.000110,0.000030,0.000141,0.000284,0.000111,0.000225,0.000189,0.000117,0.000084,0.000116,0.000089,0.000220,0.000031,0.000224,0.000171,0.000076,0.000103,0.000158,0.000101,0.000052,0.000181,0.000127,0.000154,0.000169,0.000111,0.000106,0.000110
1,1,1854221,0.000047,0.000186,0.000166,0.000184,0.000240,0.000942,0.000163,0.000264,0.000384,0.000090,0.000158,0.000140,0.000085,0.000083,0.000120,0.000207,0.000133,0.000113,0.000280,0.000167,0.000146,0.000088,0.000233,0.000194,0.000151,0.000232,0.000274,0.000142,0.025093,0.000273,0.000194,0.000109,0.000179,0.000190,0.000210,0.000099,0.000160,0.000300,0.000134,0.000293,0.000154,0.000050,0.000191,0.000187,0.000187,0.000162,0.000207,0.000164,0.000182,0.000121,0.000333,0.000175,0.000151,0.000147,0.000159,0.000162,0.000145,0.000246,...,0.000042,0.000320,0.000161,0.000139,0.000106,0.000122,0.000112,0.000098,0.000108,0.000147,0.000161,0.000194,0.000092,0.000167,0.000143,0.000154,0.000059,0.000047,0.000034,0.000142,0.000148,0.000060,0.000134,0.000266,0.000443,0.000135,0.000160,0.000203,0.000029,0.000309,0.000164,0.000205,0.000163,0.915641,0.000042,0.000199,0.000401,0.000156,0.000318,0.000266,0.000166,0.000118,0.000164,0.000125,0.000310,0.000043,0.000316,0.000241,0.000107,0.000145,0.000223,0.000143,0.000074,0.000255,0.000179,0.000218,0.000238,0.000157,0.000150,0.000155
2,2,1854224,0.000044,0.000174,0.000155,0.067575,0.000224,0.000878,0.000152,0.000246,0.000358,0.000084,0.000147,0.000131,0.000079,0.000077,0.000112,0.000193,0.000124,0.000106,0.000261,0.000156,0.000137,0.000082,0.000217,0.000181,0.000141,0.000216,0.000256,0.000133,0.000934,0.000255,0.000181,0.000102,0.000167,0.000177,0.000196,0.000092,0.000150,0.000280,0.000125,0.000273,0.000143,0.000047,0.000179,0.000175,0.000175,0.000151,0.000193,0.000153,0.000169,0.000113,0.000311,0.000163,0.000141,0.000137,0.000148,0.000151,0.000135,0.000230,...,0.000039,0.000299,0.000150,0.000130,0.000099,0.000114,0.000104,0.000091,0.000100,0.000137,0.000150,0.000181,0.000086,0.000155,0.000133,0.000143,0.000055,0.000044,0.000032,0.000133,0.000138,0.000056,0.000125,0.000248,0.000414,0.000126,0.000149,0.000189,0.000027,0.000288,0.000153,0.000191,0.000152,0.876392,0.000040,0.000185,0.000374,0.000145,0.000296,0.000248,0.000154,0.000111,0.000153,0.000117,0.000290,0.000040,0.000295,0.000225,0.000100,0.000136,0.000208,0.000133,0.000069,0.000238,0.000167,0.000203,0.000222,0.000146,0.000140,0.000145
3,3,1854233,0.000013,0.000053,0.000047,0.000052,0.000068,0.000267,0.000046,0.000075,0.409643,0.102409,0.000045,0.000040,0.000024,0.000024,0.000034,0.000059,0.000038,0.000032,0.000079,0.000047,0.000041,0.081932,0.000066,0.034183,0.000043,0.000066,0.000078,0.000040,0.000284,0.000077,0.000055,0.000031,0.000051,0.000054,0.000060,0.000028,0.000045,0.0

**In above dataframe, "index" column is the row number, "id" column is the sample/document/text id, the same with "id" in dataset, "0" to max topic number columns are the weight of each topic per document.**

### 1.2 Add Dominant Topics Column

**Find dominant topic of each document:**

In [7]:
# https://stackoverflow.com/questions/46819405/how-to-get-the-name-of-column-with-maximum-value-in-pyspark-dataframe

def argmax(cols, *args):
    return [c for c, v in zip(cols, args) if v == max(args)][0]

def add_domtopic(df):
    """
    find the dominant topic of each sample/row/document
    input: dataframe of weight of each topic
    output: the dominant topic number dataframe
    """
    argmax_udf = lambda cols: F.udf(lambda *args: argmax(cols, *args), StringType())
    return (df
            .withColumn('domtopic',
                        argmax_udf(df.columns[2:])(*df.columns[2:]))
            .withColumn('weight', 
                        F.greatest(*[F.col(x) for x in df.columns[2:-1]])))

# add the df_dominant to doc-topic matrix
df_doctopic = add_domtopic(df_doctopic)

In [9]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

### 1.3 Add Metadata Columns

**Here we only add "region" and "date" column as metadata, the accuracy of time could achieve "day" level, using the time series features we could convert "date" to year base or month base etc. depending on the need.**

In [38]:
df_doctopic = (df_doctopic
               .join(df_meta, df_doctopic.id == df_meta.id_)
               .withColumn('year', F.date_format('date', 'yyyy'))
               .drop('id_')
               .drop('date')
               .orderBy('index'))

In [11]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

### 1.4 Document - Dominant Topics Dataframe

**It is hard to plot the doc-topic matrix (high dimension), we need transform it to extract or reduce features. First we generate dominant topics dataframe which could be used to reveal the relationship between dominant topics and region/year.**

In [39]:
df_docdomtopic = (df_doctopic
                  .join(df_topics, df_doctopic.domtopic == df_topics.topic)
                  .select(F.col('id'), 
                          F.col('region'), 
                          F.col('year'), 
                          F.col('domtopic'), 
                          F.col('weight'), 
                          F.col('words'))
                  .orderBy('id'))

df_docdomtopic.cache()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

In [13]:
print('Shape of dataframe: ({}, {})'.format(df_docdomtopic.count(), len(df_docdomtopic.columns)))

df_docdomtopic.limit(5).toPandas().head()

Shape of dataframe: (1814086, 6)


,id,region,year,domtopic,weight,words
0,1854215,Auckland,1862,92,0.342072,captain vessel ship steamer boat sea board crew port water passengers island time left boats wreck ashore made cargo...
1,1854221,Auckland,1862,173,0.915641,love life heart day thy god world man eyes thou light long night earth great sweet thee death men home
2,1854224,Auckland,1862,173,0.876392,love life heart day thy god world man eyes thou light long night earth great sweet thee death men home
3,1854233,Auckland,1863,8,0.409643,business public begs orders stock notice attention inform premises street prices goods general made patronage custom...
4,1854245,Auckland,1863,14,0.301554,freight apply company passage new_zealand passengers steam london steamers tons shipping wellington agents port stea...


**Save the dataframe for later use:**

In [14]:
path = r'../models/train/domTopics'

df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

In [15]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 1.5 Average Weight Topics Dataframe

**Beside dominant topics dataframe, we could calculate average weight of each topic in a year and create dataframe for it, which could reveal the weight variety of each topic as time goes on, we could take the average weight as features and execute data mining algorithms to find patterns, e.g. correlation between features. The weights of each topic were already scaled to 0-1 by default, so the sum of average weight of each year is 1, we do not need to scale it.**

In [40]:
df_avgweight = (df_doctopic
                .drop('index')
                .drop('id')
                .drop('domtopic')
                .drop('region')
                .drop('weight')
                .groupBy('year')
                .avg()
                .orderBy('year'))

In [17]:
df_avgweight.limit(5).toPandas().head()

,year,avg(0),avg(1),avg(2),avg(3),avg(4),avg(5),avg(6),avg(7),avg(8),avg(9),avg(10),avg(11),avg(12),avg(13),avg(14),avg(15),avg(16),avg(17),avg(18),avg(19),avg(20),avg(21),avg(22),avg(23),avg(24),avg(25),avg(26),avg(27),avg(28),avg(29),avg(30),avg(31),avg(32),avg(33),avg(34),avg(35),avg(36),avg(37),avg(38),avg(39),avg(40),avg(41),avg(42),avg(43),avg(44),avg(45),avg(46),avg(47),avg(48),avg(49),avg(50),avg(51),avg(52),avg(53),avg(54),avg(55),avg(56),avg(57),avg(58),...,avg(140),avg(141),avg(142),avg(143),avg(144),avg(145),avg(146),avg(147),avg(148),avg(149),avg(150),avg(151),avg(152),avg(153),avg(154),avg(155),avg(156),avg(157),avg(158),avg(159),avg(160),avg(161),avg(162),avg(163),avg(164),avg(165),avg(166),avg(167),avg(168),avg(169),avg(170),avg(171),avg(172),avg(173),avg(174),avg(175),avg(176),avg(177),avg(178),avg(179),avg(180),avg(181),avg(182),avg(183),avg(184),avg(185),avg(186),avg(187),avg(188),avg(189),avg(190),avg(191),avg(192),avg(193),avg(194),avg(195),avg(196),avg(197),avg(198),avg(199)
0,1839,0.000006,0.000022,0.000020,0.000022,0.000029,0.000113,0.000020,0.000032,0.081538,0.000011,0.000019,0.000017,0.000010,0.000010,0.000014,0.000025,0.000016,0.000014,0.000034,0.000020,0.000018,0.000011,0.000028,0.000023,0.000018,0.000028,0.000033,0.000017,0.000121,0.006824,0.003419,0.000013,0.000022,0.000023,0.000025,0.000012,0.000019,0.000036,0.000016,0.000035,0.000019,0.000006,0.000023,0.000023,0.000023,0.000019,0.000025,0.018114,0.000022,0.000015,0.000040,0.000021,0.000018,0.000018,0.134944,0.000019,0.231617,0.000030,0.000027,...,0.000005,0.000039,0.000019,0.000017,0.000013,0.000015,0.000013,0.000012,0.000013,0.000018,0.000019,0.000023,0.000011,0.000020,0.000017,0.000019,0.000007,0.000006,0.000004,0.000017,0.000018,0.000007,0.000016,0.000032,0.000053,0.000016,0.000019,0.000024,0.000004,0.000037,0.000020,0.000025,0.000020,0.000019,0.000005,0.000024,0.013317,0.000019,0.000038,0.000032,0.000020,0.000014,0.000020,0.000015,0.000037,0.000005,0.000038,0.000029,0.000013,0.000018,0.000027,0.056043,0.000009,0.001237,0.000022,0.000026,0.000029,0.000019,0.142356,0.016906
1,1840,0.000031,0.000172,0.001180,0.000120,0.000303,0.004181,0.000263,0.000172,0.024429,0.017826,0.000146,0.002089,0.002556,0.000743,0.008423,0.000417,0.000899,0.004352,0.000246,0.000109,0.000491,0.004620,0.000814,0.001375,0.000968,0.003516,0.011927,0.000093,0.088378,0.009225,0.003816,0.000138,0.000504,0.000554,0.000868,0.001062,0.000105,0.000533,0.006938,0.003334,0.000100,0.000141,0.000365,0.000452,0.000322,0.003859,0.040511,0.002531,0.000391,0.000079,0.000302,0.013567,0.000571,0.000783,0.075627,0.000424,0.015854,0.004090,0.000274,...,0.000710,0.004263,0.000350,0.000587,0.000245,0.000080,0.000358,0.000064,0.000070,0.001645,0.007764,0.000321,0.004059,0.000369,0.001486,0.000100,0.000039,0.000031,0.000022,0.000382,0.078027,0.000040,0.011735,0.000174,0.008586,0.000088,0.002873,0.000168,0.000019,0.001249,0.039468,0.000259,0.006759,0.000102,0.000752,0.000130,0.005339,0.000236,0.000392,0.021795,0.003906,0.000078,0.000207,0.000082,0.000203,0.000129,0.003693,0.000950,0.001045,0.002640,0.001236,0.024265,0.028611,0.000525,0.001793,0.004856,0.000156,0.000518,0.001427,0.000776
2,1841,0.000967,0.001596,0.000071,0.000175,0.000801,0.002410,0.000143,0.004719,0.034793,0.022281,0.000068,0.001773,0.000068,0.000937,0.011883,0.001107,0.004352,0.007068,0.007306,0.000126,0.001817,0.000261,0.007634,0.000104,0.000065,0.001603,0.003248,0.000401,0.002710,0.008205,0.001043,0.000414,0.000160,0.000261,0.000760,0.000123,0.000454,0.009351,0.005138,0.004759,0.000370,0.000022,0.000090,0.000562,0.000081,0.001294,0.024140,0.008181,0.000546,0.000431,0.001287,0.026136,0.004137,0.002063,0.072322,0.000649,0.007378,0.001705,0.000797,...,0.000018,0.001384,0.000069,0.000060,0.000381,0.000148,0.000226,0.000396,0.000625,0.002641,0.011307,0.003079,0.001478,0.001152,0.005338,0.000223,0.000025,0.000020,0.000015,0.001256,0.085240,0.000026,0.038870,0.001573,0.018036,0.000082,0.003246,0.000578,0.000013,0.001958,0.019452,0.0

**check years are identical with dataset:**

In [62]:
year_dot = list(df_doctopic.select('year').distinct().orderBy('year').rdd.map(lambda r: r[0]).collect())
year_avg = list(df_avgweight.select('year').rdd.map(lambda r: r[0]).collect())
pd.DataFrame({'yearDocTopic':year_dot, 'yearAvgWeight':year_avg}).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
yearAvgWeight,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1907,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945
yearDocTopic,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1907,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945


**The dimension of the avrage weight dataframe is small (topic_n\*year_n), so we directly save the dataframe using Pandas:**

In [19]:
path = r'../models/train/avgWeight.csv'

df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

## 2 Prepare Dataframe for Subset

**The process for subset is the same with for full dataset, we wrapped the process to a function and call it for each subset.**

### 2.1 By Range of Time

**Generate new weight for WWI.**

In [20]:
path = r'../models/wwi/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_wwi = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_wwi = df_topics_wwi[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_wwi.shape)
#df_topics_wwi.head()

Shape of dataframe: (200, 3)


In [21]:
path = r'../models/wwi/topicKeys.csv'
df_topics_wwi.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate new weight for others (except WWI).**

In [25]:
path = r'../models/train/docTopics.txt'

START = '1912-01-01'
END = '1921-12-31'

df_doctopic = load_doctopic(path, topic_number, spark)
df_sub = df_meta.filter((df_meta['date'] < START) | (df_meta['date'] > END))
df_doctopic = df_doctopic.join(df_sub, df_doctopic.id == df_sub.id_).drop('index').drop('id').drop('id_').drop('region').drop('date')
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_others = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_others = df_topics_others[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_others.shape)
#df_topics_others.head()

Shape of dataframe: (200, 3)


,topic,weight,words
0,0,0.002454,theatre tonight exhibition universal approved adults censor recommended comedy tomorrow night plans dance george dan...
1,1,0.004981,day people procession mayor city crowd town large governor reception band morning great afternoon excellency visit p...
2,2,0.004570,election electors candidate candidates vote poll district votes elections returning elected officer council seat ele...
3,3,0.003108,oamaru aro thames dunedin north otago street waitaki john zealand town timaru kurow fleming good tyne sumpter hold g...
4,4,0.003439,christchurch timaru ashburton street canterbury akaroa lyttelton east road south john colombo bay temuka thomas burn...


In [27]:
path = r'../models/wwi/topicKeysOthers.csv'
df_topics_others.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [22]:
df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'month')

df_docdomtopic.cache()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

**Save dataframes:**

In [23]:
path = r'../models/wwi/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/wwi/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

**Convert multi files to a csv file:**

In [24]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 2.2 By Region

**Generate new weight for regions.**

In [25]:
path = r'../models/regions/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_regions = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_regions = df_topics_regions[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_regions.shape)
#df_topics_regions.head()

Shape of dataframe: (200, 3)


In [26]:
path = r'../models/regions/topicKeys.csv'
df_topics_regions.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate new weight for others (except The Regions List).**

In [4]:
path = r'../models/train/docTopics.txt'

regions = ['Auckland', 'Wellington', 'Canterbury', 'Otago']

df_doctopic = load_doctopic(path, topic_number, spark)
df_sub = df_meta.filter(F.col('region').isin(regions) == False)
df_doctopic = df_doctopic.join(df_sub, df_doctopic.id == df_sub.id_).drop('index').drop('id').drop('id_').drop('region').drop('date')
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_others = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_others = df_topics_others[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_others.shape)
#df_topics_others.head()

Shape of dataframe: (200, 3)


,topic,weight,words
0,0,0.000240,theatre tonight exhibition universal approved adults censor recommended comedy tomorrow night plans dance george dan...
1,1,0.004922,day people procession mayor city crowd town large governor reception band morning great afternoon excellency visit p...
2,2,0.004219,election electors candidate candidates vote poll district votes elections returning elected officer council seat ele...
3,3,0.000931,oamaru aro thames dunedin north otago street waitaki john zealand town timaru kurow fleming good tyne sumpter hold g...
4,4,0.001064,christchurch timaru ashburton street canterbury akaroa lyttelton east road south john colombo bay temuka thomas burn...


In [5]:
path = r'../models/regions/topicKeysOthers.csv'
df_topics_others.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [27]:
file = r'../models/regions/docTopicsInfer.txt'

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'year')

df_docdomtopic.cache()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

**Save dataframes:**

In [28]:
path = r'../models/regions/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/regions/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

**Convert multi files to a csv file:**

In [29]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 2.3 By Label

**Generate new weight for ADs.**

In [30]:
path = r'../models/ads/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_ads = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_ads = df_topics_ads[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_ads.shape)
#df_topics_ads.head()

Shape of dataframe: (200, 3)


In [31]:
path = r'../models/ads/topicKeys.csv'
df_topics_ads.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate new weight for others (except ADs).**

In [6]:
path = r'../models/train/docTopics.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_sub = df_meta.filter(F.col('ads') == False)
df_doctopic = df_doctopic.join(df_sub, df_doctopic.id == df_sub.id_).drop('index').drop('id').drop('id_').drop('region').drop('date')
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_others = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_others = df_topics_others[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_others.shape)
#df_topics_others.head()

Shape of dataframe: (200, 3)


,topic,weight,words
0,0,0.000803,theatre tonight exhibition universal approved adults censor recommended comedy tomorrow night plans dance george dan...
1,1,0.006566,day people procession mayor city crowd town large governor reception band morning great afternoon excellency visit p...
2,2,0.004722,election electors candidate candidates vote poll district votes elections returning elected officer council seat ele...
3,3,0.001844,oamaru aro thames dunedin north otago street waitaki john zealand town timaru kurow fleming good tyne sumpter hold g...
4,4,0.002184,christchurch timaru ashburton street canterbury akaroa lyttelton east road south john colombo bay temuka thomas burn...


In [7]:
path = r'../models/ads/topicKeysOthers.csv'
df_topics_others.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [32]:
file = r'../models/ads/docTopicsInfer.txt'

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'year')

df_docdomtopic.cache()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

**Save dataframes:**

In [33]:
path = r'../models/ads/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/ads/domTopics'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

**Convert multi files to a csv file:**

In [34]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

----

In [ ]:
gc.collect()